In [21]:
from IPython.display import Markdown
from google.genai import types

In [1]:
from google import genai
import time

# load api key from .env file
import dotenv
import os

In [4]:
dotenv.load_dotenv()
api_key = os.getenv("API_KEY")

In [6]:
client = genai.Client(api_key=api_key)
MODEL_ID = "gemini-2.0-flash-exp"

# Upload the video

In [14]:
video_file_name = "./assets/videos/WorldQuant BRAIN_2.mp4"

In [16]:
video_file = client.files.upload(path=video_file_name)
print(f"Completed upload : {video_file.uri}")

In [17]:
# check the state of the video whether it's processed or not
while video_file.state == "PROCESSING":
    print("Uploading file...")
    time.sleep(10)
    video_file = client.files.get(name=video_file.name)

if video_file.state == "FAILED":
    raise ValueError(video_file.state)

print(f'Video processing complete: ' + video_file.uri)

Uploading file...
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/8yj38i2re70d


In [18]:
print(video_file.state)

ACTIVE


In [25]:
prompt = "Generate a summary of the video and list all the formulas that were used and the reasoning behind it in a separate section. Just output the answer to my request and nothing else."

In [26]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents = [
        types.Content(
            role='user',
            parts=[
                types.Part.from_uri(
                    file_uri=video_file.uri,
                    mime_type=video_file.mime_type),
            ]),
            prompt,
    ]
)

In [24]:
(response.text)


'Certainly! Here\'s a summary of the video and a breakdown of the formulas with reasoning:\n\n**Video Summary**\n\nThis video by WorldQuant discusses a trading strategy based on volume data, specifically, the premise that a large current volume implies a bullish sentiment for a particular stock. It explains how this idea is implemented through the “Brain” platform using two key operators. The "ts\\_sum" operator is used to sum values in a time series for a specified number of past days, and the  “group\\_neutralize” operator  is used to neutralize this metric against different groupings like sectors. The video demonstrates how this model might work by showing different statistical data visualizations.\n\nTo avoid any sudden movements in the simulation, the presenter introduces the idea of incorporating a weighted average using a step function (ts_step). The presenter uses the step operator to give a high weighting to the current day\'s ratio and lower weight to previous day\'s ratios. 

In [27]:
Markdown('Certainly! Here\'s a summary of the video and a breakdown of the formulas with reasoning:\n\n**Video Summary**\n\nThis video by WorldQuant discusses a trading strategy based on volume data, specifically, the premise that a large current volume implies a bullish sentiment for a particular stock. It explains how this idea is implemented through the “Brain” platform using two key operators. The "ts\\_sum" operator is used to sum values in a time series for a specified number of past days, and the  “group\\_neutralize” operator  is used to neutralize this metric against different groupings like sectors. The video demonstrates how this model might work by showing different statistical data visualizations.\n\nTo avoid any sudden movements in the simulation, the presenter introduces the idea of incorporating a weighted average using a step function (ts_step). The presenter uses the step operator to give a high weighting to the current day\'s ratio and lower weight to previous day\'s ratios. The video concludes by showing detailed information from a backtest that was done for the past five years, highlighting how the model might work.\n\n**Formulas and Reasoning**\n\n1.  **First Alpha Expression:**\n\n    *   **Formula:** `group_neutralize(volume / (ts_sum(volume, 60) / 60), sector)`\n    *   **Breakdown:**\n        *   `ts_sum(volume, 60)`: Sums the stock’s trading volume over the last 60 days. This part is used to calculate an average volume over a period of 60 days. \n        *   `/ 60`: Divides the sum of volume by 60 to get the average daily volume over the last 60 days.\n        *   `volume / (ts_sum(volume, 60) / 60)`: This calculates the ratio of the current daily volume to the 60-day average volume. A higher ratio implies higher-than-average trading activity.\n        *   `group_neutralize(..., sector)`: The group_neutralize function neutralizes the alpha with a specific group, which is the sector in this case. This function ensures that the strategy’s signal is not overly dependent on the performance of a particular sector. In other words, stocks with high volumes are considered bullish within a sector.\n    *   **Reasoning:** The hypothesis is that high relative volume signals a positive market sentiment towards the stock. By using the average volume of the last 60 days, this alpha attempts to measure the current volume’s deviation from a relatively stable average, and the group_neutralize operator mitigates any sector-specific biases.\n  \n2. **Second Alpha Expression:**\n\n    *   **Formula:** `ts_step(20) * volume / (ts_sum(volume, 60) / 60)`\n    *   **Breakdown:**\n        *   `ts_step(20)`: This function creates a vector that assigns a weight of 20 to the current day, 19 to the previous day, and so on, descending to 1 for the 20th day prior.\n        *   `volume / (ts_sum(volume, 60) / 60)`: As described previously, this is the ratio of current volume to 60-day average volume.\n        *   `ts_step(20) * volume / (ts_sum(volume, 60) / 60)`: The step function is then multiplied by this volume ratio to create a weighted average of the ratio over the preceding 20 days.\n    *   **Reasoning:** This formula is used to smooth the volume ratio, by incorporating previous day’s information, the step operator ensures the signal to be less sensitive to abrupt volume changes and prevents any sudden changes in the simulation’s positions.\n\nLet me know if you have any other questions!')

Certainly! Here's a summary of the video and a breakdown of the formulas with reasoning:

**Video Summary**

This video by WorldQuant discusses a trading strategy based on volume data, specifically, the premise that a large current volume implies a bullish sentiment for a particular stock. It explains how this idea is implemented through the “Brain” platform using two key operators. The "ts\_sum" operator is used to sum values in a time series for a specified number of past days, and the  “group\_neutralize” operator  is used to neutralize this metric against different groupings like sectors. The video demonstrates how this model might work by showing different statistical data visualizations.

To avoid any sudden movements in the simulation, the presenter introduces the idea of incorporating a weighted average using a step function (ts_step). The presenter uses the step operator to give a high weighting to the current day's ratio and lower weight to previous day's ratios. The video concludes by showing detailed information from a backtest that was done for the past five years, highlighting how the model might work.

**Formulas and Reasoning**

1.  **First Alpha Expression:**

    *   **Formula:** `group_neutralize(volume / (ts_sum(volume, 60) / 60), sector)`
    *   **Breakdown:**
        *   `ts_sum(volume, 60)`: Sums the stock’s trading volume over the last 60 days. This part is used to calculate an average volume over a period of 60 days. 
        *   `/ 60`: Divides the sum of volume by 60 to get the average daily volume over the last 60 days.
        *   `volume / (ts_sum(volume, 60) / 60)`: This calculates the ratio of the current daily volume to the 60-day average volume. A higher ratio implies higher-than-average trading activity.
        *   `group_neutralize(..., sector)`: The group_neutralize function neutralizes the alpha with a specific group, which is the sector in this case. This function ensures that the strategy’s signal is not overly dependent on the performance of a particular sector. In other words, stocks with high volumes are considered bullish within a sector.
    *   **Reasoning:** The hypothesis is that high relative volume signals a positive market sentiment towards the stock. By using the average volume of the last 60 days, this alpha attempts to measure the current volume’s deviation from a relatively stable average, and the group_neutralize operator mitigates any sector-specific biases.
  
2. **Second Alpha Expression:**

    *   **Formula:** `ts_step(20) * volume / (ts_sum(volume, 60) / 60)`
    *   **Breakdown:**
        *   `ts_step(20)`: This function creates a vector that assigns a weight of 20 to the current day, 19 to the previous day, and so on, descending to 1 for the 20th day prior.
        *   `volume / (ts_sum(volume, 60) / 60)`: As described previously, this is the ratio of current volume to 60-day average volume.
        *   `ts_step(20) * volume / (ts_sum(volume, 60) / 60)`: The step function is then multiplied by this volume ratio to create a weighted average of the ratio over the preceding 20 days.
    *   **Reasoning:** This formula is used to smooth the volume ratio, by incorporating previous day’s information, the step operator ensures the signal to be less sensitive to abrupt volume changes and prevents any sudden changes in the simulation’s positions.

Let me know if you have any other questions!